# 0. 드라이브 마운트

드라이브에 저장된 텍스트 파일을 활용하기 위해 우선 드라이브를 마운트한다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


해당 파일이 있는 폴더의 위치를 확인 후 폴더 내에 있는 모든 파일을 뽑아온다.

In [ ]:
import os
data_dir = '/content/drive/MyDrive/data'
os.listdir(data_dir)

['kowiki', 'corpus']

압축시킨 해당 파일의 경로를 하나로 정리한다.

In [ ]:
import os
corpus = os.path.join(data_dir, "corpus", "corpus.zip")
corpus_dir = os.path.dirname(corpus)
corpus

'/content/drive/MyDrive/data/corpus/corpus.zip'

corpus 파일의 내용을 일부 확인한다.

In [ ]:
import zipfile
with zipfile.ZipFile(corpus) as z:
    with z.open('corpus.txt') as f:
        for i, line in enumerate(f):
            if i >= 50:
                break
            line = line.decode('utf-8').strip()
            print(line)

아내가 드디어 출산하게 되어서 정말 신이 나.
아내분이 출산을 하시는군요. 정말 축하드려요.
아 지금 정말 신이 나.
잘 된 일이네요.
아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.
좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.
당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
약 종류가 많아 번거로우시겠어요.
건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.
현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?
약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.
약을 잘 챙겨 먹어 건강해지시길 바랄게요.
고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.
아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.
이 상황에서 할 수 있는 게 어떤 것이 있을까요?
중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.
수업을 어떤 방식으로 열심히 듣는게 좋을까요?
재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
재취업 후 첫 월급이라 정말 기쁘시겠어요.
퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 수 있어서 너무 행복해.
이 기분을 유지하려면 어떻게 해야 할까요?
회사생활을 열심히 해서 계속 월급을 받을거야!
계속 원하시는 대로 회사생활이 이어지길 바랄게요.
빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.
기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?
빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.
빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.
""
""
이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를 하고. 그러면 둘이 하루 종일 함께 있겠네. 생각만 해도 숨이 막혀.
그러시군요. 은퇴 후 남편분과 함께 집에 있을 생각을 하니 막막한 기분이 드시는군요.
둘이서 같이 뭘 해본 적이 없

# 1. sentencepiece 라이브러리를 활용한 말뭉치 단어 사전 개발

- sentencepiece 라이브러리 설치

sentencepiece 라이브러리는 BPE알고리즘을 지원하며, 단어 사전의 크기를 지정할 수 있다.

In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 2.7 MB/s 


vocab_size가 5000개인 vocab 모델을 구현한다.

In [ ]:
import sentencepiece as spm
def train_sentencepiece(corpus, prefix, vocab_size=5000):
    """
    sentencepiece를 이용해 vocab 학습
    :param corpus: 학습할 말뭉치
    :param prefix: 저장할 vocab 이름
    :param vocab_size: vocab 개수
    """
    spm.SentencePieceTrainer.train(
        f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +  
        " --model_type=unigram" +
        " --max_sentence_length=999999" +  
        " --pad_id=0 --pad_piece=[PAD]" +  
        " --unk_id=1 --unk_piece=[UNK]" +  
        " --bos_id=2 --bos_piece=[BOS]" +  
        " --eos_id=3 --eos_piece=[EOS]" +  
        " --user_defined_symbols=[SEP],[CLS],[MASK]" +  
        " --input_sentence_size=100000" +  
        " --shuffle_input_sentence=true")  

현재 작업하고 있는 폴더에 corpus.txt를 압축한 파일을 푼다.

In [ ]:
!unzip /content/drive/MyDrive/data/corpus/corpus.zip
print(os.listdir("./"))

Archive:  /content/drive/MyDrive/data/corpus/corpus.zip
  inflating: corpus.txt              
['.config', 'drive', 'corpus.txt', 'sample_data']


sentencepiece를 이용하여 말뭉치 사전을 만든다.

In [ ]:
train_sentencepiece("corpus.txt", "corpus_5000")

말뭉치 사전이 잘 만들어졌는지 확인해본다.

In [ ]:
print(os.listdir("./"))

['.config', 'corpus_5000.vocab', 'corpus_5000.model', 'drive', 'corpus.txt', 'sample_data']


shutil모듈의 copy 함수로 만들어진 사전 파일을 복사시킨다.

In [ ]:
import shutil
corpus_dir = os.path.dirname(corpus)
shutil.copy("corpus_5000.model", corpus_dir)
shutil.copy("corpus_5000.vocab", corpus_dir)
shutil.copy("corpus.txt", corpus_dir)

print(os.listdir(corpus_dir))

['corpus.zip', 'corpus_5000.model', 'corpus_5000.vocab', 'corpus.txt', 'corpus_mecab_5000.model', 'corpus_mecab_5000.vocab', 'corpus-mecab.txt']


model 파일과 vocab 파일이 생성되었는데, 사용할 때는 model 파일을 사용한다.

In [ ]:
spm_vocab = spm.SentencePieceProcessor()
spm_vocab.load(os.path.join(corpus_dir, "corpus_5000.model"))

True

말뭉치 사전을 기준으로 해당 텍스트가 어떻게 토큰화되었는지 확인해본다.

In [ ]:
with zipfile.ZipFile(corpus) as z:
    with z.open('corpus.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            line = line.decode('utf-8').strip()
            print(line)
            tokens = spm_vocab.encode_as_pieces(line)
            print(tokens)
            _ids = spm_vocab.encode_as_ids(line)
            print(_ids)

아내가 드디어 출산하게 되어서 정말 신이 나.
['▁아내', '가', '▁드', '디어', '▁출산', '하게', '▁되어서', '▁정말', '▁신이', '▁나', '.']
[113, 10, 330, 1238, 790, 118, 787, 46, 2197, 27, 7]
아내분이 출산을 하시는군요. 정말 축하드려요.
['▁아내분이', '▁출산', '을', '▁하시는군요', '.', '▁정말', '▁축하드려요', '.']
[1758, 790, 9, 109, 7, 46, 1279, 7]
아 지금 정말 신이 나.
['▁아', '▁지금', '▁정말', '▁신이', '▁나', '.']
[412, 64, 46, 2197, 27, 7]
잘 된 일이네요.
['▁잘', '▁된', '▁일이네요', '.']
[28, 413, 3708, 7]
아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.
['▁아기', '가', '▁점', '점', '▁클', '게', '▁벌써', '▁기대', '가', '▁되네', '.', '▁내가', '▁많이', '▁놀아', '줘야', '지', '.']
[1446, 10, 438, 435, 3390, 32, 942, 633, 10, 1772, 7, 34, 31, 2951, 1475, 25, 7]


# 2. Mecab으로 형태소 분석 후 말뭉치 단어 사전 개발

- 형태소 분석기 Mecab 설치

mecab은 형태소 분석기 중 가장 많이 사용되고 있다.

In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4 MB 47.6 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 448 kB 50.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab() 

morph(하나의 형태소) 단위로 분할시킨 말뭉치 파일 corpus-mecab 텍스트 파일을 생성한다.

In [ ]:
from tqdm.notebook import tqdm, trange
import zipfile

with open("corpus-mecab.txt", "w") as o_f:
    with zipfile.ZipFile(corpus) as z:
        with z.open('corpus.txt') as f:
            for i, line in enumerate(tqdm(f)):
                line = line.decode('utf-8').strip()
                tokens = mecab.morphs(line)
                string = " ".join(tokens)
                o_f.write(string)
                o_f.write("\n")

0it [00:00, ?it/s]

corpus-mecab 파일이 제대로 생성됐는지 확인한다.

In [ ]:
print(os.listdir("./"))

['.config', 'corpus_5000.vocab', 'corpus_5000.model', 'drive', 'corpus-mecab.txt', 'corpus.txt', 'sample_data']


생성된 파일을 복사해서 백업시킨다.

In [ ]:
import shutil
corpus_dir = os.path.dirname(corpus)
shutil.copy("corpus-mecab.txt", corpus_dir)

'/content/drive/MyDrive/data/corpus/corpus-mecab.txt'

mecab 형태소 분석기로 분석한 corpus를 확인한다.

In [ ]:
with open(os.path.join(corpus_dir,'corpus-mecab.txt')) as f:
    for i, line in enumerate(f):
        if i > 5:
            break
        print(line.strip())

아내 가 드디어 출산 하 게 되 어서 정말 신 이 나 .
아 내분 이 출산 을 하 시 는군요 . 정말 축하 드려요 .
아 지금 정말 신 이 나 .
잘 된 일 이 네요 .
아기 가 점점 클 게 벌써 기대 가 되 네 . 내 가 많이 놀 아 줘야지 .
좋 은 아빠 가 되 실 거 같 아요 . 진심 으로 축하 드려요 .


mecab을 사용한 vocab_size가 5000인 사전 파일을 만든다.

In [ ]:
train_sentencepiece(os.path.join(corpus_dir,'corpus-mecab.txt'), "corpus_mecab_5000")

사전 파일을 잘 만들어졌는지 확인한다.

In [ ]:
print(os.listdir("./"))

['.config', 'corpus_5000.vocab', 'corpus_5000.model', 'drive', 'corpus_mecab_5000.model', 'corpus-mecab.txt', 'corpus_mecab_5000.vocab', 'corpus.txt', 'sample_data']


만들어진 사전 파일도 복사 후 백업시킨다.

In [ ]:
corpus_dir = os.path.dirname(corpus)
shutil.copy("corpus_mecab_5000.model", corpus_dir)
shutil.copy("corpus_mecab_5000.vocab", corpus_dir)

print(os.listdir(corpus_dir))

['corpus.zip', 'corpus_5000.model', 'corpus_5000.vocab', 'corpus.txt', 'corpus_mecab_5000.model', 'corpus_mecab_5000.vocab', 'corpus-mecab.txt']


생성된 사전을 사용하기 위해 사전을 로드시킨다.

In [ ]:
spm_morph_vocab = spm.SentencePieceProcessor()
spm_morph_vocab.load(os.path.join(corpus_dir, "corpus_mecab_5000.model"))

True

mecab으로 전처리한 말뭉치 사전으로 corpus가 어떻게 토큰화되었는지 확인해본다.

In [ ]:
with zipfile.ZipFile(corpus) as z:
    with z.open('corpus.txt') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            print(line)
            tokens = spm_morph_vocab.encode_as_pieces(line)
            print(tokens)
            _ids = spm_morph_vocab.encode_as_ids(line)
            print(_ids)

b'\xec\x95\x84\xeb\x82\xb4\xea\xb0\x80 \xeb\x93\x9c\xeb\x94\x94\xec\x96\xb4 \xec\xb6\x9c\xec\x82\xb0\xed\x95\x98\xea\xb2\x8c \xeb\x90\x98\xec\x96\xb4\xec\x84\x9c \xec\xa0\x95\xeb\xa7\x90 \xec\x8b\xa0\xec\x9d\xb4 \xeb\x82\x98.\r\n'
[b'\xe2\x96\x81', b'\xec\x95\x84\xeb\x82\xb4', b'\xea\xb0\x80', b'\xe2\x96\x81\xeb\x93\x9c', b'\xeb\x94\x94', b'\xec\x96\xb4', b'\xe2\x96\x81\xec\xb6\x9c\xec\x82\xb0', b'\xed\x95\x98', b'\xea\xb2\x8c', b'\xe2\x96\x81\xeb\x90\x98', b'\xec\x96\xb4\xec\x84\x9c', b'\xe2\x96\x81\xec\xa0\x95\xeb\xa7\x90', b'\xe2\x96\x81\xec\x8b\xa0', b'\xec\x9d\xb4', b'\xe2\x96\x81\xeb\x82\x98', b'.']
[7, 157, 102, 201, 404, 193, 788, 60, 41, 50, 1419, 111, 169, 37, 20, 338]
b'\xec\x95\x84\xeb\x82\xb4\xeb\xb6\x84\xec\x9d\xb4 \xec\xb6\x9c\xec\x82\xb0\xec\x9d\x84 \xed\x95\x98\xec\x8b\x9c\xeb\x8a\x94\xea\xb5\xb0\xec\x9a\x94. \xec\xa0\x95\xeb\xa7\x90 \xec\xb6\x95\xed\x95\x98\xeb\x93\x9c\xeb\xa0\xa4\xec\x9a\x94.\r\n'
[b'\xe2\x96\x81', b'\xec\x95\x84\xeb\x82\xb4', b'\xeb\xb6\x84', b'\xec

# 3. 1번과 2번의 단어사전 비교하기

비교하기 위해 앞에서 만든 vocab size가 5000인 사전과 mecab으로 전처리한 vocab size가 5000인 사전을 업로드한다.

In [ ]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load(os.path.join(data_dir,'corpus', "corpus_5000.model"))
sp_morph = spm.SentencePieceProcessor()
sp_morph.load(os.path.join(data_dir,'corpus', "corpus_mecab_5000.model"))

True

In [ ]:
text = """많이 서운하셨겠어요.
친구들에게 서운함을 느꼈을 때 해결하는 방법이 있나요?
카페에 가서 대화하거나 같이 술 마시면서 이야기했던 것 같아.
대화로 문제를 해결해오셨던 거군요?
그렇다면 이번 일에는 어떻게 하실 예정인가요?
직접 서운한 감정을 친구에게 얘기하려고 해.
이번에도 대화를 통해 불편한 마음을 풀려고 하시는군요.
"""

In [ ]:
print(text)

많이 서운하셨겠어요.
친구들에게 서운함을 느꼈을 때 해결하는 방법이 있나요?
카페에 가서 대화하거나 같이 술 마시면서 이야기했던 것 같아.
대화로 문제를 해결해오셨던 거군요?
그렇다면 이번 일에는 어떻게 하실 예정인가요?
직접 서운한 감정을 친구에게 얘기하려고 해.
이번에도 대화를 통해 불편한 마음을 풀려고 하시는군요.



해당 문장을 1번 말뭉치 사전으로 토큰화시킨 결과다.

In [ ]:
print(spm_vocab.encode_as_pieces(text))

['▁많이', '▁서운하', '셨겠어요', '.', '▁친구들에게', '▁서운함', '을', '▁느꼈', '을', '▁때', '▁해결하는', '▁방법이', '▁있나요', '?', '▁카페', '에', '▁가서', '▁대화', '하거나', '▁같이', '▁술', '▁마시면', '서', '▁이야기', '했던', '▁것', '▁같아', '.', '▁대화로', '▁문제', '를', '▁해결해', '오', '셨던', '▁거군요', '?', '▁그렇다면', '▁이번', '▁일에', '는', '▁어떻게', '▁하실', '▁예정', '인가요', '?', '▁직접', '▁서운한', '▁감정', '을', '▁친구에게', '▁얘기', '하려고', '▁해', '.', '▁이번에', '도', '▁대화', '를', '▁통해', '▁불편한', '▁마음', '을', '▁풀려', '고', '▁하시는군요', '.']


해당 문장을 2번 말뭉치 사전으로 토큰화시킨 결과다.

In [ ]:
print(spm_morph_vocab.encode_as_pieces(text))

['▁많', '이', '▁', '서운', '하', '셨', '겠어', '요', '.', '▁친', '구', '들', '에게', '▁', '서운', '함', '을', '▁느꼈', '을', '▁때', '▁해결', '하', '는', '▁방', '법', '이', '▁', '있', '나', '요', '?', '▁카페', '에', '▁가', '서', '▁대화', '하', '거', '나', '▁같', '이', '▁술', '▁마', '시', '면', '서', '▁이야', '기', '했', '던', '▁', '것', '▁같', '아', '.', '▁대화', '로', '▁문제', '를', '▁해결', '해', '오', '셨', '던', '▁거', '군요', '?', '▁그렇', '다면', '▁이', '번', '▁일', '에', '는', '▁어떻', '게', '▁하', '실', '▁예정', '인가', '요', '?', '▁직', '접', '▁', '서운', '한', '▁감', '정', '을', '▁친', '구', '에게', '▁얘', '기', '하', '려고', '▁해', '.', '▁이', '번', '에', '도', '▁대화', '를', '▁통해', '▁불', '편', '한', '▁마음', '을', '▁풀려', '고', '▁하', '시', '는군요', '.']


맨 앞문장 '많이 서운하셨겠어요'로 비교했을 때 1번 사전은 '많이, 서운하, 셨겠어요'와 같이 3개의 형태소로 구분되었고, 2번 사전은 '많, 이, 서운, 하, 셨, 겠어, 요'로 7개로 구분되었다.
이로써 같은 문장으로 두 사전을 비교했을 때 형태소 분석기 mecab으로 전처리시켰던 말뭉치 사전이 더 정교하게 토큰화된 것을 알 수 있었다. 